In [ ]:
# Cell 0: Consolidated Imports and Auto-Logger Bootstrap
# 
# Purpose: All imports consolidated here following PEP 8 order
# Initialize basic logging - all other setup in Cell 1

# ============================================================================
# CONSOLIDATED IMPORTS - ALL IMPORTS FOR THE NOTEBOOK
# ============================================================================

# Standard library imports (alphabetical order)
import importlib
import importlib.util
import json
import os
import pickle
import re
import signal
import sys
import time
import traceback
import uuid
import warnings

# Standard library from imports (alphabetical order)
from collections import OrderedDict
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager
from datetime import datetime
from functools import wraps
from pathlib import Path
from typing import Dict, List, Optional, Any, Set, Tuple

# Third-party imports (alphabetical order)
import edgar
import numpy as np
import psycopg2
import requests
import torch

# Third-party from imports (alphabetical order by module)
from bs4 import BeautifulSoup
from edgar import Filing, find, set_identity, Company
from edgar.documents import parse_html
from edgar.documents.extractors.section_extractor import SectionExtractor
from huggingface_hub import login
from IPython import get_ipython
from ipykernel.iostream import OutStream
from psycopg2 import pool
from psycopg2.extras import execute_values
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    pipeline,
    BitsAndBytesConfig
)

# Environment imports
from kaggle_secrets import UserSecretsClient

# ============================================================================
# AUTO-LOGGER BOOTSTRAP (USING CONSOLIDATED IMPORTS)
# ============================================================================

# Get GitHub token for logger access
user_secrets = UserSecretsClient()
GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")

print("🔧 Setting up consolidated imports and logger bootstrap...")

# Clone/update repo for logger access
REPO_URL = f"https://{GITHUB_TOKEN}@github.com/amiralpert/SmartReach.git"
LOCAL_PATH = "/kaggle/working/SmartReach"

if os.path.exists(LOCAL_PATH):
    !cd {LOCAL_PATH} && git pull origin main > /dev/null 2>&1
else:
    !git clone {REPO_URL} {LOCAL_PATH} > /dev/null 2>&1

# Add to path
if f'{LOCAL_PATH}/BizIntel' not in sys.path:
    sys.path.insert(0, f'{LOCAL_PATH}/BizIntel')

# Initialize logger with minimal setup
logger_path = f"{LOCAL_PATH}/BizIntel/Scripts/KaggleLogger/auto_logger.py"
if os.path.exists(logger_path):
    spec = importlib.util.spec_from_file_location("auto_logger", logger_path)
    auto_logger = importlib.util.module_from_spec(spec)
    sys.modules["auto_logger"] = auto_logger
    spec.loader.exec_module(auto_logger)
    
    # Simple logger setup - database manager will be provided by Cell 1
    logger = None  # Will be properly initialized after Cell 1 runs
    print("✅ Auto-logger module loaded")
else:
    logger = None
    print("⚠️  Logger module not found - continuing without logging")

print("✅ Cell 0: All imports consolidated (33+ imports) + bootstrap complete")
print("   📦 Standard library: importlib, json, os, pickle, re, signal, sys, time, etc.")
print("   🔗 Third-party: edgar, numpy, psycopg2, requests, torch, transformers, bs4")
print("   🌐 Environment: kaggle_secrets")

In [ ]:
# Cell 1: GitHub Setup and Simplified Configuration

# Install required packages first
!pip install edgartools transformers torch accelerate huggingface_hub requests beautifulsoup4 'lxml[html_clean]' uuid numpy newspaper3k --quiet
!pip install -U bitsandbytes --quiet

# ============================================================================
# CENTRALIZED CONFIGURATION - Simplified
# ============================================================================

# Use Kaggle secrets for all sensitive credentials
user_secrets = UserSecretsClient()

# Database Configuration (using Kaggle secrets for security)
NEON_CONFIG = {
    'host': user_secrets.get_secret("NEON_HOST"),
    'database': user_secrets.get_secret("NEON_DATABASE"),
    'user': user_secrets.get_secret("NEON_USER"), 
    'password': user_secrets.get_secret("NEON_PASSWORD"),
    'sslmode': 'require'
}

# Master Configuration Dictionary - Simplified
CONFIG = {
    # GitHub Settings
    'github': {
        'token': user_secrets.get_secret("GITHUB_TOKEN"),
        'repo_url': f"https://{user_secrets.get_secret('GITHUB_TOKEN')}@github.com/amiralpert/SmartReach.git",
        'local_path': "/kaggle/working/SmartReach"
    },
    
    # Database Settings
    'database': NEON_CONFIG,
    
    # Model Configuration
    'models': {
        'confidence_threshold': 0.8, # confidence NER model needs to store an entity 
        'batch_size': 16, # number of text chunks to process at one time through NER
        'max_length': 512, # max chunk size to process through NER 512 token limit for BERT models 
        'chunk_overlap': 0.1,  # 10% overlap between chunks for complete entity extraction
        'warm_up_enabled': True,
        'warm_up_text': 'Pfizer announced FDA approval for new cancer drug targeting BRCA mutations.'
    },
    
    # Cache Settings - Simplified
    'cache': {
        'enabled': True,
        'max_size_mb': 100,  # Maximum cache size in MB
        'eviction_policy': 'LRU'  # Least Recently Used
    },
    
    # Processing Settings - Simplified  
    'processing': {
        'filing_batch_size': 1,
        'filing_query_limit': 1,       # Explicit limit for get_unprocessed_filings()
        'enable_relationships': True,   # Enable/disable relationship extraction
        'entity_batch_size': 10000,    # Max entities per database insert
        'section_validation': True,    # Enforce section name validation
        'debug_mode': False,
        'max_insert_batch': 50000,     # Maximum batch for database inserts
        'deduplication_threshold': 0.85
    },
    
    # Llama 3.1 Configuration
    'llama': {
        'enabled': True,
        'model_name': 'meta-llama/Llama-3.1-8B-Instruct',
        'batch_size': 15,              # Entities per Llama call (for future batching)
        'max_new_tokens': 50,          # Reduced from 200 for speed
        'context_window': 400,         # Reduced from 1000 chars for speed  
        'temperature': 0.3,            # Sampling temperature
        'entity_context_window': 400,  # Reduced from 500 chars for entity context
        'test_max_tokens': 50,         # For model testing
        'min_confidence_filter': 0.8,  # Entity filtering threshold
        'timeout_seconds': 30,         # Timeout for model calls
    },
    
    # EdgarTools Settings
    'edgar': {
        'identity': "SmartReach BizIntel amir@leanbio.consulting"
    }
}

if not CONFIG['github']['token']:
    raise ValueError("❌ GITHUB_TOKEN is required in Kaggle secrets")

if not CONFIG['database']['password']:
    raise ValueError("❌ NEON_PASSWORD is required in Kaggle secrets")

print("✅ Configuration loaded from Kaggle secrets")
print(f"   Database: {CONFIG['database']['host']}")
print(f"   Processing: Filing batch={CONFIG['processing']['filing_batch_size']}, Query limit={CONFIG['processing']['filing_query_limit']}")
print(f"   Llama 3.1: Enabled={CONFIG['llama']['enabled']}, Tokens={CONFIG['llama']['max_new_tokens']}, Context={CONFIG['llama']['context_window']}")
print(f"   Cache: Max size={CONFIG['cache']['max_size_mb']}MB")
print(f"   Relationships: {'Enabled' if CONFIG['processing']['enable_relationships'] else 'Disabled'}")

# ============================================================================
# ERROR LOGGING FUNCTIONS
# ============================================================================

def log_error(component: str, message: str, exception: Exception = None, context: dict = None):
    """Enhanced error logging with stack traces"""
    if exception:
        error_msg = f"ERROR [{component}]: {message} - {type(exception).__name__}: {str(exception)}"
        # Add stack trace for debugging
        if CONFIG['processing'].get('debug_mode', False):
            error_msg += f"
Stack trace:
{traceback.format_exc()}"
    else:
        error_msg = f"ERROR [{component}]: {message}"
    
    if context:
        error_msg += f" | Context: {context}"
    
    print(error_msg)  # Auto-logger captures this
    return error_msg

def log_warning(component: str, message: str, context: dict = None):
    """Standardized warning logging"""
    warning_msg = f"WARNING [{component}]: {message}"
    if context:
        warning_msg += f" | Context: {context}"
    print(warning_msg)
    return warning_msg

def log_info(component: str, message: str):
    """Standardized info logging"""
    info_msg = f"INFO [{component}]: {message}"
    print(info_msg)
    return info_msg

# ============================================================================
# SIMPLE DATABASE CONNECTION
# ============================================================================

@contextmanager
def get_db_connection():
    """Simple database connection context manager"""
    conn = None
    try:
        conn = psycopg2.connect(**CONFIG['database'])
        yield conn
        conn.commit()
    except Exception as e:
        if conn:
            conn.rollback()
        raise e
    finally:
        if conn:
            conn.close()

# ============================================================================
# SIZE-LIMITED LRU CACHE
# ============================================================================

class SizeLimitedLRUCache:
    """LRU cache with size limit in MB"""
    
    def __init__(self, max_size_mb: int):
        self.max_size_bytes = max_size_mb * 1024 * 1024
        self.cache = OrderedDict()
        self.current_size = 0
        self.hits = 0
        self.misses = 0
    
    def _estimate_size(self, value: str) -> int:
        """Estimate size of cached value in bytes"""
        return len(value.encode('utf-8')) if isinstance(value, str) else sys.getsizeof(value)
    
    def get(self, key: str):
        """Get item from cache"""
        if key in self.cache:
            self.hits += 1
            # Move to end (most recently used)
            self.cache.move_to_end(key)
            return self.cache[key]
        self.misses += 1
        return None
    
    def put(self, key: str, value, size: int = None):
        """Put item in cache with LRU eviction"""
        if size is None:
            size = self._estimate_size(value)
        
        # Remove old entries if needed
        while self.current_size + size > self.max_size_bytes and self.cache:
            evicted_key, evicted_value = self.cache.popitem(last=False)
            self.current_size -= self._estimate_size(evicted_value)
            log_info("Cache", f"Evicted {evicted_key} to maintain size limit")
        
        # Add new entry
        if key in self.cache:
            self.current_size -= self._estimate_size(self.cache[key])
        
        self.cache[key] = value
        self.current_size += size
        self.cache.move_to_end(key)
    
    def get_stats(self) -> dict:
        """Get cache statistics"""
        hit_rate = (self.hits / (self.hits + self.misses) * 100) if (self.hits + self.misses) > 0 else 0
        return {
            'entries': len(self.cache),
            'size_mb': self.current_size / (1024 * 1024),
            'hits': self.hits,
            'misses': self.misses,
            'hit_rate': hit_rate
        }

# Initialize global cache for EdgarTools sections
SECTION_CACHE = SizeLimitedLRUCache(CONFIG['cache']['max_size_mb'])

# ============================================================================
# GITHUB SETUP
# ============================================================================

print("
📦 Setting up GitHub repository...")
local_path = CONFIG['github']['local_path']
repo_url = CONFIG['github']['repo_url']

# Clone or update repo with force pull
if os.path.exists(local_path):
    log_info("GitHub", f"Repository exists at {local_path}")
    log_info("GitHub", "Force updating from main branch")
    !cd {local_path} && git fetch origin
    !cd {local_path} && git reset --hard origin/main
    !cd {local_path} && git pull origin main
    log_info("GitHub", "Repository updated successfully")
    
    # Show current commit
    !cd {local_path} && echo "Current commit:" && git log --oneline -1
else:
    log_info("GitHub", f"Cloning repository to {local_path}")
    !git clone {repo_url} {local_path}
    log_info("GitHub", "Repository cloned successfully")

# Clear any cached modules from previous runs
modules_to_clear = [key for key in sys.modules.keys() if 'auto_logger' in key.lower() or 'clean' in key.lower()]
for mod in modules_to_clear:
    del sys.modules[mod]
    log_info("ModuleCache", f"Cleared cached module: {mod}")

# Add to Python path for regular imports
if f'{local_path}/BizIntel' in sys.path:
    sys.path.remove(f'{local_path}/BizIntel')
sys.path.insert(0, f'{local_path}/BizIntel')

log_info("Setup", "Python path configured for SEC entity extraction")

# Configure EdgarTools authentication - REQUIRED by SEC
set_identity(CONFIG['edgar']['identity'])
log_info("EdgarTools", f"Identity configured: {CONFIG['edgar']['identity']}")

print("🚀 SEC ENTITY EXTRACTION ENGINE INITIALIZED - SIMPLIFIED AND CLEAN")
print("="*80)
print(f"✅ GitHub: Repository ready at {local_path}")
print(f"✅ Database: Connected to {CONFIG['database']['host']}")
print(f"✅ Cache: {CONFIG['cache']['max_size_mb']}MB LRU cache initialized")
print(f"✅ EdgarTools: Identity set for SEC compliance")
print("="*80)

In [3]:
# Cell 2: Database Functions and ORM-like Models with Batching - SIMPLIFIED

# Basic startup check - restart kernel if issues persist
print("Starting Cell 2 - EdgarTools section extraction")

# Ensure identity is set
set_identity(CONFIG['edgar']['identity'])

# ============================================================================
# TIMEOUT HANDLER FOR EDGARTOOLS API CALLS - FIX FOR HANGING
# ============================================================================

class TimeoutError(Exception):
    """Custom timeout exception"""
    pass

def timeout_handler(signum, frame):
    """Signal handler for timeout"""
    raise TimeoutError("EdgarTools API call timed out")

def with_timeout(seconds=30):
    """Decorator to add timeout to functions"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set the signal alarm
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                # Disable the alarm
                signal.alarm(0)
            return result
        return wrapper
    return decorator

# ============================================================================
# PROBLEMATIC FILINGS TO SKIP
# ============================================================================

# Known problematic filings that cause indefinite hangs
PROBLEMATIC_FILINGS = [
    '0001699031-25-000166',  # Grail 10-Q that caused 11+ hour hang
]

# ============================================================================
# TIMEOUT-WRAPPED EDGARTOOLS CALLS
# ============================================================================

@with_timeout(30)  # 30 second timeout
def find_filing_with_timeout(accession_number: str):
    """Find filing with timeout protection"""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting EdgarTools find() for {accession_number}")
    filing = find(accession_number)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] find() completed successfully")
    return filing

@with_timeout(60)  # 60 second timeout for HTML download
def get_html_with_timeout(filing):
    """Get HTML content with timeout protection"""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting html() fetch...")
    html_content = filing.html()
    if html_content:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] html() completed, size: {len(html_content):,} bytes")
    else:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] html() returned empty content")
    return html_content

@with_timeout(30)  # 30 second timeout for parsing
def parse_html_with_timeout(html_content):
    """Parse HTML with timeout protection"""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting HTML parsing...")
    document = parse_html(html_content)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] HTML parsing completed")
    return document

def get_filing_sections(accession_number: str, filing_type: str = None) -> Dict[str, str]:
    """Get structured sections from SEC filing using accession number
    
    ENHANCED: With timeouts, progress monitoring, and problematic filing skipping
    """
    # Skip known problematic filings
    if accession_number in PROBLEMATIC_FILINGS:
        log_warning("EdgarTools", f"Skipping known problematic filing: {accession_number}")
        return {}
    
    # Check cache first
    cache_key = f"{accession_number}#{filing_type or 'UNKNOWN'}"
    cached_sections = SECTION_CACHE.get(cache_key)
    if cached_sections:
        log_info("Cache", f"Cache hit for {accession_number}")
        return cached_sections
    
    try:
        # Find filing with timeout protection
        try:
            filing = find_filing_with_timeout(accession_number)
        except TimeoutError:
            log_error("EdgarTools", f"Timeout finding filing {accession_number} (30s exceeded)")
            return {}
        
        if not filing:
            raise ValueError(f"Filing not found for accession: {accession_number}")
            
        # Auto-detect filing type if not provided
        if not filing_type:
            filing_type = getattr(filing, 'form', '10-K')
        
        log_info("EdgarTools", f"Found {filing_type} for {getattr(filing, 'company', 'Unknown Company')}")
        
        # Get structured HTML content with timeout
        try:
            html_content = get_html_with_timeout(filing)
        except TimeoutError:
            log_error("EdgarTools", f"Timeout fetching HTML for {accession_number} (60s exceeded)")
            return {}
        
        if not html_content:
            raise ValueError("No HTML content available")
        
        # Limit HTML size to prevent memory issues
        MAX_HTML_SIZE = 10 * 1024 * 1024  # 10MB limit
        if len(html_content) > MAX_HTML_SIZE:
            log_warning("EdgarTools", f"HTML too large ({len(html_content):,} bytes), truncating to {MAX_HTML_SIZE:,}")
            html_content = html_content[:MAX_HTML_SIZE]
        
        # Parse HTML to Document object with timeout
        try:
            document = parse_html_with_timeout(html_content)
        except TimeoutError:
            log_error("EdgarTools", f"Timeout parsing HTML for {accession_number} (30s exceeded)")
            return {}
        
        # Extract sections using SectionExtractor
        extractor = SectionExtractor(filing_type=filing_type)
        sections = extractor.extract(document)
        
        log_info("EdgarTools", f"SectionExtractor found {len(sections)} sections")
        
        # Convert sections to text dictionary
        section_texts = {}
        for section_name, section in sections.items():
            try:
                if hasattr(section, 'text'):
                    text = section.text() if callable(section.text) else section.text
                    if isinstance(text, str) and text.strip():
                        section_texts[section_name] = text.strip()
                        print(f"      • {section_name}: {len(text):,} chars")
                elif hasattr(section, '__str__'):
                    text = str(section).strip()
                    if text:
                        section_texts[section_name] = text
                        print(f"      • {section_name}: {len(text):,} chars (via str)")
            except Exception as section_e:
                log_warning("EdgarTools", f"Could not extract section {section_name}", {"error": str(section_e)})
                continue
        
        # If SectionExtractor returns no sections, fall back to full document text
        if not section_texts:
            log_warning("EdgarTools", "No structured sections found, using full document fallback")
            full_text = document.text() if hasattr(document, 'text') and callable(document.text) else str(document)
            if full_text and len(full_text.strip()) > 100:  # Only use if substantial content
                # Limit full document size
                if len(full_text) > MAX_HTML_SIZE:
                    log_warning("EdgarTools", f"Full document too large ({len(full_text):,} chars), truncating")
                    full_text = full_text[:MAX_HTML_SIZE]
                section_texts['full_document'] = full_text.strip()
                log_info("EdgarTools", f"Using full document: {len(full_text):,} chars")
        
        # Cache the result
        if section_texts and CONFIG['cache']['enabled']:
            SECTION_CACHE.put(cache_key, section_texts)
            log_info("Cache", f"Cached sections for {accession_number} ({len(section_texts)} sections)")
        
        return section_texts
        
    except Exception as e:
        log_error("EdgarTools", f"Failed to fetch filing {accession_number}", e)
        return {}  # Return empty dict on network/API failure

def route_sections_to_models(sections: Dict[str, str], filing_type: str) -> Dict[str, List[str]]:
    """Route sections to appropriate NER models based on filing type"""
    routing = {
        'biobert': [],
        'bert_base': [],
        'roberta': [],
        'finbert': []
    }
    
    if filing_type.upper() in ['10-K', '10-Q']:
        for section_name, section_text in sections.items():
            # FinBERT gets financial statements exclusively
            if 'financial' in section_name.lower() or 'statement' in section_name.lower():
                routing['finbert'].append(section_name)
            else:
                # All other sections go to BERT/RoBERTa/BioBERT
                routing['bert_base'].append(section_name)
                routing['roberta'].append(section_name)
                routing['biobert'].append(section_name)
    
    elif filing_type.upper() == '8-K':
        # 8-K: all item sections go to all four models
        for section_name in sections.keys():
            routing['biobert'].append(section_name)
            routing['bert_base'].append(section_name)
            routing['roberta'].append(section_name)
            routing['finbert'].append(section_name)
    
    else:
        # Default routing for other filing types
        for section_name in sections.keys():
            routing['bert_base'].append(section_name)
            routing['roberta'].append(section_name)
            routing['biobert'].append(section_name)
    
    # Remove empty routing
    routing = {model: sections_list for model, sections_list in routing.items() if sections_list}
    
    return routing

def process_sec_filing_with_sections(filing_data: Dict) -> Dict:
    """Process SEC filing with section-based extraction
    
    ENHANCED: With timeout protection and progress monitoring
    """
    try:
        filing_id = filing_data.get('id')
        accession_number = filing_data.get('accession_number')  # DIRECT FROM DATABASE
        filing_type = filing_data.get('filing_type', '10-K')
        company_domain = filing_data.get('company_domain', 'Unknown')
        filing_url = filing_data.get('url')  # Still keep for reference
        
        log_info("FilingProcessor", f"Processing {filing_type} for {company_domain}")
        print(f"   📄 Filing ID: {filing_id}")
        print(f"   📑 Accession: {accession_number}")
        
        # Validate accession number
        if not accession_number:
            raise ValueError(f"Missing accession number for filing {filing_id}")
        
        # Check if this is a problematic filing
        if accession_number in PROBLEMATIC_FILINGS:
            log_warning("FilingProcessor", f"Skipping problematic filing: {accession_number}")
            return {
                'filing_id': filing_id,
                'company_domain': company_domain,
                'filing_type': filing_type,
                'accession_number': accession_number,
                'error': 'Skipped - known problematic filing',
                'processing_status': 'skipped'
            }
        
        # Get structured sections using accession directly
        sections = get_filing_sections(accession_number, filing_type)
        if not sections:
            raise ValueError("No sections extracted")
        
        log_info("FilingProcessor", f"Extracted {len(sections)} sections")
        
        # Route sections to models
        model_routing = route_sections_to_models(sections, filing_type)
        print(f"   🎯 Model routing: {[f'{model}: {len(secs)} sections' for model, secs in model_routing.items()]}")
        
        # Validate section names if configured
        if CONFIG['processing']['section_validation']:
            missing_sections = [name for name in sections.keys() if not name]
            if missing_sections:
                log_warning("FilingProcessor", f"Found {len(missing_sections)} sections without names")
        
        # Show cache statistics
        cache_stats = SECTION_CACHE.get_stats()
        if cache_stats['hits'] > 0:
            print(f"   📊 Cache: {cache_stats['hit_rate']:.1f}% hit rate, {cache_stats['size_mb']:.1f}MB used")
        
        return {
            'filing_id': filing_id,
            'company_domain': company_domain,
            'filing_type': filing_type,
            'accession_number': accession_number,
            'url': filing_url,
            'sections': sections,
            'model_routing': model_routing,
            'total_sections': len(sections),
            'processing_status': 'success'
        }
        
    except TimeoutError as e:
        log_error("FilingProcessor", "Filing processing timed out", e, 
                 {"filing_id": filing_data.get('id'), "accession": filing_data.get('accession_number')})
        return {
            'filing_id': filing_data.get('id'),
            'company_domain': filing_data.get('company_domain', 'Unknown'),
            'filing_type': filing_data.get('filing_type', 'Unknown'),
            'accession_number': filing_data.get('accession_number'),
            'error': 'Processing timeout',
            'processing_status': 'timeout'
        }
    except Exception as e:
        log_error("FilingProcessor", "Filing processing failed", e, 
                 {"filing_id": filing_data.get('id'), "accession": filing_data.get('accession_number')})
        return {
            'filing_id': filing_data.get('id'),
            'company_domain': filing_data.get('company_domain', 'Unknown'),
            'filing_type': filing_data.get('filing_type', 'Unknown'),
            'accession_number': filing_data.get('accession_number'),
            'error': str(e),
            'processing_status': 'failed'
        }

def get_unprocessed_filings(limit: int = 5) -> List[Dict]:
    """Get SEC filings that haven't been processed yet
    
    ENHANCED: Skip known problematic filings
    """
    with get_db_connection() as conn:  # PHASE 2: Using context manager
        cursor = conn.cursor()
        
        # Build exclusion list for SQL
        exclusion_list = "', '".join(PROBLEMATIC_FILINGS)
        exclusion_clause = f"AND sf.accession_number NOT IN ('{exclusion_list}')" if PROBLEMATIC_FILINGS else ""
        
        cursor.execute(f"""
            SELECT 
                sf.id, 
                sf.company_domain, 
                sf.filing_type, 
                sf.accession_number,
                sf.url, 
                sf.filing_date, 
                sf.title
            FROM raw_data.sec_filings sf
            LEFT JOIN system_uno.sec_entities_raw ser 
                ON ser.sec_filing_ref = CONCAT('SEC_', sf.id)
            WHERE sf.accession_number IS NOT NULL  -- Must have accession
                AND ser.sec_filing_ref IS NULL     -- Not yet processed
                {exclusion_clause}                 -- Skip problematic filings
            ORDER BY sf.filing_date DESC
            LIMIT %s
        """, (limit,))
        
        filings = cursor.fetchall()
        cursor.close()
        
        log_info("DatabaseQuery", f"Retrieved {len(filings)} unprocessed filings (excluded {len(PROBLEMATIC_FILINGS)} problematic)")
        
        return [{
            'id': filing[0],
            'company_domain': filing[1],
            'filing_type': filing[2],
            'accession_number': filing[3],
            'url': filing[4],
            'filing_date': filing[5],
            'title': filing[6]
        } for filing in filings]

# Test the simplified extraction with timeout protection
log_info("Test", "Starting section extraction test with timeout protection")

test_filings = get_unprocessed_filings(limit=1)

if test_filings:
    print(f"
🧪 Testing with filing: {test_filings[0]['company_domain']} - {test_filings[0]['filing_type']}")
    print(f"   Accession: {test_filings[0]['accession_number']}")
    
    test_result = process_sec_filing_with_sections(test_filings[0])
    
    if test_result['processing_status'] == 'success':
        log_info("Test", f"✅ Successfully extracted {test_result['total_sections']} sections")
    elif test_result['processing_status'] == 'timeout':
        log_warning("Test", f"⏱️ Processing timed out - filing may be too large or slow")
    elif test_result['processing_status'] == 'skipped':
        log_info("Test", f"⏭️ Skipped problematic filing")
    else:
        log_error("Test", f"❌ Section extraction failed: {test_result.get('error')}")
else:
    log_info("Test", "No test filings available (all may be processed or problematic)")

print("✅ Cell 2 complete - EdgarTools section extraction with timeout protection ready")

In [ ]:
# Cell 3: Optimized Entity Extraction Pipeline - Uses Cell 2's Pre-Extracted Sections

print("🚀 Loading Optimized EntityExtractionPipeline (Handler Classes Eliminated)...")

class EntityExtractionPipeline:
    """Streamlined entity extraction using Cell 2's pre-processed sections and routing"""
    
    def __init__(self, config: Dict):
        self.config = config
        self.models = {}
        self.stats = {
            "entities_extracted": 0,
            "entities_filtered": 0,
            "sections_processed": 0,
            "filings_processed": 0
        }
        
        # Essential filtering from CONFIG
        self._biobert_skip_categories = {'0'}  # Skip BioBERT category "0"
        self._finbert_common_words = {'the', 'and', 'or', 'but', 'company', 'inc', 'corporation', 'corp'}
        self._bert_skip_misc = True  # Skip BERT MISC category
        
        # Map Cell 2's routing names to our model names
        self._routing_to_model_map = {
            'biobert': 'biobert',
            'bert_base': 'bert',      # Cell 2 uses 'bert_base'
            'roberta': 'roberta', 
            'finbert': 'finbert'
        }
        
        self._load_models()
    
    def _load_models(self):
        """Load NER models efficiently"""
        model_configs = [
            ('biobert', 'alvaroalon2/biobert_diseases_ner'),
            ('bert', 'dslim/bert-base-NER'),
            ('finbert', 'ProsusAI/finbert'), 
            ('roberta', 'Jean-Baptiste/roberta-large-ner-english')
        ]
        
        # Determine device
        device = -1  # CPU by default
        if torch.cuda.is_available():
            device = 0
            print("   🚀 Using GPU acceleration")
        else:
            print("   💻 Using CPU (GPU not available)")
        
        for name, model_id in model_configs:
            try:
                self.models[name] = pipeline(
                    "ner",
                    model=model_id,
                    aggregation_strategy="average",
                    device=device
                )
                print(f"      ✓ {name} loaded")
            except Exception as e:
                print(f"      ❌ Failed to load {name}: {e}")
        
        print(f"   ✅ Loaded {len(self.models)} NER models")
        
        # Warm up models if enabled
        if self.config.get('models', {}).get('warm_up_enabled', False):
            self._warm_up_models()
    
    def _warm_up_models(self):
        """Warm up models with test text"""
        test_text = self.config.get('models', {}).get('warm_up_text', 'Test entity extraction.')
        print("   🔥 Warming up models...")
        
        for name, model in self.models.items():
            try:
                model(test_text)
                print(f"      ✓ {name} warmed up")
            except Exception as e:
                print(f"      ⚠️ {name} warm-up failed: {e}")
    
    def process_filing_entities(self, filing_data: Dict) -> List[Dict]:
        """Main function: Extract entities using Cell 2's section extraction and routing"""
        
        # Step 1: Use Cell 2's section extraction function directly
        section_result = process_sec_filing_with_sections(filing_data)
        
        if section_result['processing_status'] != 'success':
            print(f"   ❌ Section extraction failed: {section_result.get('error', 'Unknown')}")
            return []
        
        # Step 2: Extract entities using Cell 2's sections and routing
        entities = self._extract_entities_from_sections(section_result)
        
        self.stats['filings_processed'] += 1
        self.stats['entities_extracted'] += len(entities)
        
        print(f"   ✅ Extracted {len(entities)} entities from {section_result['total_sections']} sections")
        
        return entities
    
    def _extract_entities_from_sections(self, section_result: Dict) -> List[Dict]:
        """Extract entities using Cell 2's sections and model routing"""
        sections = section_result['sections']
        model_routing = section_result['model_routing']
        
        all_entities = []
        self.stats['sections_processed'] += len(sections)
        
        # Process each model's assigned sections (using Cell 2's routing)
        for routing_model_name, assigned_section_names in model_routing.items():
            
            # Map Cell 2's model name to our model name
            our_model_name = self._routing_to_model_map.get(routing_model_name)
            
            if not our_model_name or our_model_name not in self.models:
                print(f"      ⚠️ Model '{routing_model_name}' -> '{our_model_name}' not available")
                continue
            
            print(f"      🔄 Processing {len(assigned_section_names)} sections with {our_model_name}")
            
            # Extract entities from each assigned section
            for section_name in assigned_section_names:
                section_text = sections.get(section_name)
                if not section_text:
                    continue
                
                section_entities = self._extract_from_single_section(
                    section_text, our_model_name, section_name, section_result
                )
                all_entities.extend(section_entities)
        
        # Merge overlapping entities
        merged_entities = self._merge_entities(all_entities)
        
        return merged_entities
    
    def _extract_from_single_section(self, section_text: str, model_name: str, 
                                   section_name: str, section_result: Dict) -> List[Dict]:
        """Extract entities from single section with essential filtering"""
        try:
            # Extract raw entities
            raw_entities = self.models[model_name](section_text)
            
            filtered_entities = []
            for entity in raw_entities:
                # Apply confidence threshold
                if entity['score'] < self.config.get('models', {}).get('confidence_threshold', 0.5):
                    continue
                
                entity_text = entity['word'].strip()
                entity_category = entity['entity_group']
                
                # Apply essential model-specific filtering
                if not self._passes_essential_filters(model_name, entity_text, entity_category):
                    self.stats['entities_filtered'] += 1
                    continue
                
                filtered_entities.append({
                    'extraction_id': str(uuid.uuid4()),
                    'company_domain': section_result['company_domain'],
                    'entity_text': entity_text,
                    'entity_category': self._normalize_entity_type(entity_category),
                    'confidence_score': float(entity['score']),
                    'character_start': entity['start'],
                    'character_end': entity['end'],
                    'section_name': section_name,
                    'sec_filing_ref': f"SEC_{section_result['filing_id']}",
                    'primary_model': model_name,
                    'filing_type': section_result['filing_type'],
                    'filing_date': section_result.get('filing_date'),
                    'accession_number': section_result['accession_number'],
                    'model_source': model_name,
                    'surrounding_text': self._get_surrounding_text(section_text, entity['start'], entity['end']),
                    'data_source': 'sec_filings',
                    'extraction_timestamp': datetime.now()
                })
            
            return filtered_entities
            
        except Exception as e:
            print(f"      ❌ Entity extraction failed with {model_name} on {section_name}: {e}")
            return []
    
    def _passes_essential_filters(self, model_name: str, entity_text: str, entity_category: str) -> bool:
        """Essential filtering logic per model (consolidated from handler classes)"""
        entity_lower = entity_text.lower()
        
        # Essential filtering based on model
        if model_name == 'biobert':
            # Skip BioBERT category "0" (non-medical text misclassified)
            return entity_category not in self._biobert_skip_categories
        
        elif model_name == 'finbert':
            # Skip common words for FinBERT
            return entity_lower not in self._finbert_common_words
        
        elif model_name == 'bert':
            # Skip BERT MISC category if configured
            return not (self._bert_skip_misc and entity_category == 'MISC')
        
        # RoBERTa and others: minimal filtering
        return len(entity_text) >= 2
    
    def _normalize_entity_type(self, entity_type: str) -> str:
        """Normalize entity types across models"""
        mappings = {
            'Disease': 'MEDICAL_CONDITION',
            'Chemical': 'MEDICATION',
            'Drug': 'MEDICATION',
            'PER': 'PERSON',
            'ORG': 'ORGANIZATION', 
            'LOC': 'LOCATION',
            'MONEY': 'FINANCIAL',
            'PERCENT': 'FINANCIAL'
        }
        return mappings.get(entity_type, entity_type.upper())
    
    def _get_surrounding_text(self, section_text: str, start: int, end: int, window: int = 100) -> str:
        """Get surrounding text for context"""
        text_start = max(0, start - window)
        text_end = min(len(section_text), end + window)
        return section_text[text_start:text_end]
    
    def _merge_entities(self, entities: List[Dict]) -> List[Dict]:
        """Simplified entity merging - highest confidence wins"""
        if not entities:
            return []
        
        # Group by position within same section and filing
        position_groups = {}
        for entity in entities:
            key = (entity['sec_filing_ref'], entity['section_name'], 
                  entity['character_start'], entity['character_end'])
            position_groups.setdefault(key, []).append(entity)
        
        # Merge logic: take highest confidence entity from each group
        merged = []
        for group in position_groups.values():
            if len(group) == 1:
                # Single entity - keep as is
                entity = group[0]
                entity['is_merged'] = False
                entity['models_detected'] = [entity['primary_model']]
                merged.append(entity)
            else:
                # Multiple entities at same position - merge
                best_entity = max(group, key=lambda x: x['confidence_score'])
                best_entity['is_merged'] = True
                best_entity['models_detected'] = [e['primary_model'] for e in group]
                best_entity['all_confidences'] = {e['primary_model']: e['confidence_score'] for e in group}
                merged.append(best_entity)
        
        return merged
    
    def get_extraction_stats(self) -> Dict:
        """Get extraction statistics"""
        return {
            'models_loaded': len(self.models),
            'filings_processed': self.stats['filings_processed'],
            'sections_processed': self.stats['sections_processed'],
            'entities_extracted': self.stats['entities_extracted'],
            'entities_filtered': self.stats['entities_filtered'],
            'filter_rate': f"{(self.stats['entities_filtered'] / max(1, self.stats['entities_extracted'] + self.stats['entities_filtered']) * 100):.1f}%"
        }

# Initialize the entity extraction pipeline
entity_pipeline = EntityExtractionPipeline(CONFIG)

print(f"✅ EntityExtractionPipeline initialized:")
stats = entity_pipeline.get_extraction_stats()
for key, value in stats.items():
    print(f"   • {key}: {value}")

print("✅ Cell 3 complete - Optimized entity extraction ready (handler classes eliminated)")

In [ ]:
# Cell 4: Relationship Extractor with Local Llama 3.1-8B

print("🚀 Loading In-Memory Pipeline with Storage and Local Llama 3.1-8B...")

# ================================================================================
# RELATIONSHIP EXTRACTOR WITH LOCAL LLAMA 3.1-8B
# ================================================================================

class RelationshipExtractor:
    """Extract company-centric relationships using local Llama 3.1-8B"""
    
    def __init__(self, llama_config: Dict = None):
        """Initialize with local Llama 3.1-8B model"""
        self.config = llama_config or CONFIG.get('llama', {})
        self.model = None
        self.tokenizer = None
        self.stats = {
            'entities_processed': 0,
            'relationships_found': 0,
            'llama_calls': 0,
            'processing_time': 0
        }
        
                
        # Verify CONFIG is available
        if not CONFIG.get('llama', {}).get('enabled', False):
            print("   ⚠️ Llama configuration disabled in CONFIG")
            return
        
        try:
            # Auto-login to HuggingFace using Kaggle secret
            print("   🔐 Logging in to HuggingFace...")
            user_secrets = UserSecretsClient()
            hf_token = user_secrets.get_secret('HUGGINGFACE_TOKEN')
            
            if hf_token:
                login(token=hf_token, add_to_git_credential=False)
                print("   ✅ Logged in to HuggingFace")
            else:
                print("   ⚠️ No HUGGINGFACE_TOKEN found in Kaggle secrets")
                return
            
            # Configure 4-bit quantization for memory efficiency
            print("   ⚙️ Configuring 4-bit quantization...")
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16
            )
            
            # Load Llama 3.1-8B model
            print("   📥 Loading Llama 3.1-8B-Instruct (this may take a minute)...")
            model_name = CONFIG["llama"]["model_name"]
            
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True
            )
            
            print("   ✅ Llama 3.1-8B loaded successfully (4-bit quantized)")
            
            # Test the model
            test_messages = [
                {"role": "user", "content": "What is a partnership? Answer in one sentence."}
            ]
            test_input = self.tokenizer.apply_chat_template(test_messages, return_tensors="pt", tokenize=True)
            
            with torch.no_grad():
                outputs = self.model.generate(test_input, max_new_tokens=CONFIG["llama"]["test_max_tokens"], temperature=CONFIG["llama"]["temperature"])
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"   🧪 Test response: {response[:100]}...")
            
        except Exception as e:
            print(f"   ❌ Failed to load Llama 3.1-8B: {e}")
            print("   ⚠️ Relationship extraction will be disabled")
            self.model = None
            self.tokenizer = None
    
    def extract_company_relationships(self, 
                                     entities: List[Dict], 
                                     sections: Dict[str, str],
                                     company_domain: str) -> List[Dict]:
        """Extract relationships between company and all found entities"""
        if not self.model or not self.tokenizer or not entities:
            return []
        
        print(f"   🔍 Analyzing relationships for {company_domain}")
        
        # Trust Cell 3's filtering - entities are already high-quality
        
        relationships = []
        
        # Group entities by section for context efficiency
        entities_by_section = {}
        for entity in entities:
            section = entity.get('section_name', 'unknown')
            if section not in entities_by_section:
                entities_by_section[section] = []
            entities_by_section[section].append(entity)
        
        # Process each section's entities
        # Progress tracking initialization
        total_entities_to_process = sum(len(ents) for ents in entities_by_section.values())
        entities_processed_count = 0
        start_time = time.time()
        print(f"      🎯 Total entities to analyze: {total_entities_to_process}")

        for section_name, section_entities in entities_by_section.items():
            if section_name not in sections:
                continue
                
            section_text = sections[section_name]
            section_relationships_count = 0  # Initialize counter for this section
            print(f"      📑 Processing {len(section_entities)} entities in '{section_name}'")
            
            # Process entities in batches for efficiency
            batch_size = CONFIG.get('llama', {}).get('batch_size', 5)  # Default 5 entities per batch
            
            # Filter out self-references first
            filtered_entities = []
            company_name = company_domain.replace('.com', '').replace('tx', '')
            
            for entity in section_entities:
                if entity['entity_text'].lower() != company_name.lower():
                    context = self._get_entity_context(entity, section_text)
                    filtered_entities.append((entity, context, section_name))
            
            if not filtered_entities:
                continue
            
            print(f"         🔬 Processing {len(filtered_entities)} entities in batches of {batch_size}")
            
            # Process entities in batches
            for batch_start in range(0, len(filtered_entities), batch_size):
                batch_end = min(batch_start + batch_size, len(filtered_entities))
                entities_batch = filtered_entities[batch_start:batch_end]
                
                entities_processed_count += len(entities_batch)
                
                # Show progress
                progress_pct = (entities_processed_count * 100) // total_entities_to_process
                print(f"         ⏳ Batch {batch_start//batch_size + 1}: Processing entities {batch_start+1}-{batch_end} ({progress_pct}% complete)")
                
                # Process batch with Llama
                batch_relationships = self._analyze_relationship_batch(entities_batch)
                
                if batch_relationships:
                    relationships.extend(batch_relationships)
                    section_relationships_count += len(batch_relationships)
                    self.stats['relationships_found'] += len(batch_relationships)
                
                self.stats['entities_processed'] += len(entities_batch)
        
        print(f"   ✅ Found {len(relationships)} relationships from {len(entities)} entities")
        return relationships
    
    def _get_entity_context(self, entity: Dict, section_text: str, window: int = None) -> str:
        """Get context around an entity"""
        if window is None:
            window = CONFIG["llama"]["entity_context_window"]
        start = max(0, entity.get('character_start', entity.get('char_start', 0)) - window)
        end = min(len(section_text), entity.get('character_end', entity.get('char_end', 0)) + window)
        return section_text[start:end]
    
    def _analyze_relationship_batch(self, entities_batch: List[Tuple[Dict, str, str]]) -> List[Dict]:
        """Analyze multiple entities in a single Llama call for efficiency"""
        if not self.model or not self.tokenizer or not entities_batch:
            return []
        
        try:
            # Build batch prompt for multiple entities
            prompt = f"""You are an expert at analyzing business relationships from SEC filings.

Analyze the business relationships for the following entities and provide detailed semantic extraction.

ENTITIES TO ANALYZE:
"""
            
            # Add each entity to the prompt
            for i, (entity, context, section_name) in enumerate(entities_batch, 1):
                company_domain = entity.get("company_domain", "Unknown")
                prompt += f"""
Entity {i}:
- Company: {company_domain}
- Entity: {entity["entity_text"]} (Type: {entity.get("entity_category", "UNKNOWN")})
- Section: {section_name}
- Context: {context[:400]}

"""
            
            prompt += """
For EACH entity, extract the following information and respond in JSON format:

{
  "entity_1": {
    "relationship_type": "<PARTNERSHIP|COMPETITOR|REGULATORY|CLINICAL_TRIAL|SUPPLIER|CUSTOMER|INVESTOR|ACQUISITION|LICENSING|RESEARCH|NONE>",
    "semantic_action": "<initiated|expanded|milestone_reached|terminated|ongoing>",
    "semantic_impact": "<positive|negative|neutral|mixed>",
    "semantic_tags": ["tag1", "tag2", "tag3"],
    "monetary_value": "<number_or_null>",
    "percentage_value": "<number_or_null>",
    "duration_months": "<number_or_null>",
    "entity_count": "<number_or_null>",
    "mentioned_time_period": "<Q1 2024|2025|next year|etc>",
    "temporal_precision": "<EXACT_DATE|QUARTER|YEAR|RELATIVE>",
    "confidence_level": "<high|medium|low>",
    "summary": "<one_sentence_summary>",
    "business_impact_summary": "<detailed_3_sentence_analysis>",
    "regulatory_implications": "<regulatory_analysis_or_none>",
    "competitive_implications": "<competitive_analysis_or_none>"
  },
  "entity_2": { ... },
  ...
}

EXTRACTION GUIDELINES:
- monetary_value: Extract dollar amounts as numbers (e.g., "M" → 50000000)
- percentage_value: Extract percentages as numbers (e.g., "45%" → 45.0)
- duration_months: Convert time periods to months (e.g., "3 years" → 36)
- entity_count: Extract numerical counts (e.g., "three trials" → 3)
- semantic_tags: Key biotech/business terms like ["oncology", "phase_2", "FDA", "partnership"]
- temporal_precision: How precise is the time reference
- Set fields to null if not mentioned in context

Return valid JSON only, no additional text."""

            # Create messages for chat format
            messages = [
                {"role": "system", "content": "You are an expert at analyzing business relationships from SEC filings. Always respond with valid JSON in the exact format requested."},
                {"role": "user", "content": prompt}
            ]
            
            # Apply chat template
            inputs = self.tokenizer.apply_chat_template(
                messages,
                return_tensors="pt",
                tokenize=True
            )
            
            # Generate response with expanded token limit
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_new_tokens=2000,  # Increased from 50 to 2000
                    temperature=CONFIG["llama"]["temperature"],
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            # Decode response
            llama_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extract just the assistant's response
            if "assistant" in llama_response:
                llama_response = llama_response.split("assistant")[-1].strip()
            
            self.stats['llama_calls'] += 1
            
            # Parse JSON response
            return self._parse_batch_llama_response(llama_response, entities_batch)
            
        except Exception as e:
            print(f"         ⚠️ Batch Llama analysis failed: {e}")
            return []

    def _parse_batch_llama_response(self, response: str, entities_batch: List) -> List[Dict]:
        """Parse JSON response from batch Llama analysis"""
        try:
            import json
            
            # Clean up response to extract JSON
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            if json_start >= 0 and json_end > json_start:
                json_str = response[json_start:json_end]
                
                # Parse JSON
                batch_results = json.loads(json_str)
                
                relationships = []
                for i, (entity, context, section_name) in enumerate(entities_batch, 1):
                    entity_key = f"entity_{i}"
                    if entity_key in batch_results:
                        result = batch_results[entity_key]
                        
                        # Skip if no relationship found
                        if result.get('relationship_type') == 'NONE':
                            continue
                        
                        # Build relationship record with rich metadata
                        relationship = {
                            'company_domain': entity.get('company_domain', ''),
                            'entity_text': entity['entity_text'],
                            'entity_type': entity.get('entity_category', 'UNKNOWN'),
                            'entity_id': entity.get('extraction_id', str(uuid.uuid4())),
                            'relationship_type': result.get('relationship_type', 'UNKNOWN'),
                            'semantic_action': result.get('semantic_action', 'ongoing'),
                            'semantic_impact': result.get('semantic_impact', 'neutral'),
                            'semantic_tags': result.get('semantic_tags', []),
                            'monetary_value': result.get('monetary_value'),
                            'percentage_value': result.get('percentage_value'),
                            'duration_months': result.get('duration_months'),
                            'entity_count': result.get('entity_count'),
                            'mentioned_time_period': result.get('mentioned_time_period', ''),
                            'temporal_precision': result.get('temporal_precision', 'RELATIVE'),
                            'business_impact': result.get('semantic_impact', 'neutral'),
                            'confidence_level': result.get('confidence_level', 'medium'),
                            'summary': result.get('summary', ''),
                            'business_impact_summary': result.get('business_impact_summary', ''),
                            'regulatory_implications': result.get('regulatory_implications', ''),
                            'competitive_implications': result.get('competitive_implications', ''),
                            'section_name': section_name,
                            'context_used': context[:500],
                            'llama_response': response[:1000],
                            'extraction_timestamp': datetime.now().isoformat()
                        }
                        
                        relationships.append(relationship)
                
                return relationships
                
        except Exception as e:
            print(f"Failed to parse batch Llama response: {e}")
            return []
        
        return []
    def _parse_llama_response(self, response: str) -> Optional[Dict]:
        """Parse structured response from Llama"""
        try:
            lines = response.strip().split('\n')
            parsed = {}
            
            for line in lines:
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip().upper()
                    value = value.strip()
                    
                    if key == 'TYPE':
                        parsed['type'] = value
                    elif key == 'DIRECTION':
                        parsed['direction'] = value
                    elif key == 'IMPACT':
                        parsed['impact'] = value.lower()
                    elif key == 'CONFIDENCE':
                        parsed['confidence'] = value.lower()
                    elif key == 'SUMMARY':
                        parsed['summary'] = value
            
            # Validate required fields
            required = ['type', 'direction', 'impact', 'confidence', 'summary']
            if all(field in parsed for field in required):
                return parsed
            
            return None
            
        except Exception:
            return None

# ================================================================================
# SOPHISTICATED SEMANTIC RELATIONSHIP STORAGE
# ================================================================================


class SemanticRelationshipStorage:
    """Advanced storage system using sophisticated semantic relationship schema"""
    
    def __init__(self, db_config: Dict):
        self.db_config = db_config
        self.storage_stats = {
            'buckets_created': 0,
            'buckets_updated': 0,
            'events_stored': 0,
            'sessions_tracked': 0,
            'transactions_completed': 0,
            'transactions_failed': 0
        }
    
    def store_relationships_with_buckets(self, relationships: List[Dict], filing_ref: str, session_id: str = None) -> bool:
        """Store relationships using bucket aggregation pattern"""
        if not relationships:
            return True
        
        conn = None
        try:
            conn = psycopg2.connect(**self.db_config)
            cursor = conn.cursor()
            conn.autocommit = False
            
            print(f"   💾 Storing {len(relationships)} relationships using sophisticated schema...")
            
            for relationship in relationships:
                # Step 1: Find or create bucket for this company-entity-type combination
                bucket_id = self._find_or_create_bucket(cursor, relationship)
                
                # Step 2: Store semantic event with rich metadata
                self._store_semantic_event(cursor, relationship, bucket_id, filing_ref, session_id)
                
                # Step 3: Update bucket aggregation
                self._update_bucket_aggregation(cursor, bucket_id, relationship)
            
            conn.commit()
            self.storage_stats['transactions_completed'] += 1
            self.storage_stats['events_stored'] += len(relationships)
            
            print(f"      ✅ Stored {len(relationships)} relationship events")
            return True
            
        except Exception as e:
            if conn:
                conn.rollback()
            self.storage_stats['transactions_failed'] += 1
            print(f"      ❌ Failed to store relationships: {e}")
            return False
        finally:
            if conn:
                conn.close()
    
    def _find_or_create_bucket(self, cursor, relationship: Dict) -> str:
        """Find existing bucket or create new one for company-entity-type combination"""
        company_domain = relationship['company_domain']
        entity_name = relationship['entity_text']
        relationship_type = relationship['relationship_type']
        
        # Try to find existing bucket
        cursor.execute("""
            SELECT bucket_id FROM system_uno.relationship_buckets
            WHERE company_domain = %s AND entity_name = %s AND relationship_type = %s
        """, (company_domain, entity_name, relationship_type))
        
        result = cursor.fetchone()
        if result:
            return result[0]  # Return existing bucket_id
        
        # Create new bucket
        cursor.execute("""
            INSERT INTO system_uno.relationship_buckets
            (company_domain, entity_name, relationship_type, master_semantic_summary, 
             first_mentioned_date, last_mentioned_date, total_mentions, is_active)
            VALUES (%s, %s, %s, %s, CURRENT_DATE, CURRENT_DATE, 1, TRUE)
            RETURNING bucket_id
        """, (company_domain, entity_name, relationship_type, relationship.get('summary', '')))
        
        bucket_id = cursor.fetchone()[0]
        self.storage_stats['buckets_created'] += 1
        return bucket_id
    
    def _store_semantic_event(self, cursor, relationship: Dict, bucket_id: str, filing_ref: str, session_id: str = None):
        """Store individual relationship event with semantic metadata"""
        cursor.execute("""
            INSERT INTO system_uno.relationship_semantic_events
            (bucket_id, source_entity_id, sec_filing_ref, filing_date, filing_type, section_name,
             semantic_summary, semantic_action, semantic_impact, semantic_tags,
             monetary_value, percentage_value, duration_months, entity_count,
             mentioned_time_period, temporal_precision, 
             business_impact_summary, regulatory_implications, competitive_implications,
             original_context_snippet, confidence_score, llama_prompt_version, event_timestamp)
            VALUES (%s, %s, %s, CURRENT_DATE, '10-K', %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, '2.0', CURRENT_TIMESTAMP)
        """, (
            bucket_id,
            relationship.get('entity_id'),
            filing_ref,
            relationship.get('section_name', ''),
            relationship.get('summary', ''),
            relationship.get('semantic_action', 'ongoing'),
            relationship.get('semantic_impact', 'neutral'),
            relationship.get('semantic_tags', []),
            relationship.get('monetary_value'),
            relationship.get('percentage_value'),
            relationship.get('duration_months'),
            relationship.get('entity_count'),
            relationship.get('mentioned_time_period', ''),
            relationship.get('temporal_precision', 'RELATIVE'),
            relationship.get('business_impact_summary', ''),
            relationship.get('regulatory_implications', ''),
            relationship.get('competitive_implications', ''),
            relationship.get('context_used', '')[:500],  # Limit to 500 chars
            float(relationship.get('confidence_level', 0.5)) if isinstance(relationship.get('confidence_level'), (int, float)) else 0.5,
        ))
    
    def _update_bucket_aggregation(self, cursor, bucket_id: str, relationship: Dict):
        """Update bucket aggregated metrics"""
        cursor.execute("""
            UPDATE system_uno.relationship_buckets
            SET 
                last_mentioned_date = CURRENT_DATE,
                total_mentions = total_mentions + 1,
                updated_at = CURRENT_TIMESTAMP,
                master_semantic_summary = %s
            WHERE bucket_id = %s
        """, (relationship.get('summary', ''), bucket_id))
        
        self.storage_stats['buckets_updated'] += 1
    
    def create_analysis_session(self, company_domain: str, filing_batch: List[str]) -> str:
        """Create analysis session for tracking Llama processing"""
        try:
            conn = psycopg2.connect(**self.db_config)
            cursor = conn.cursor()
            
            cursor.execute("""
                INSERT INTO system_uno.semantic_analysis_sessions
                (company_domain, filing_batch, primary_prompt_version, session_start)
                VALUES (%s, %s, '1.0', CURRENT_TIMESTAMP)
                RETURNING session_id
            """, (company_domain, filing_batch))
            
            session_id = cursor.fetchone()[0]
            conn.commit()
            cursor.close()
            conn.close()
            
            self.storage_stats['sessions_tracked'] += 1
            return session_id
            
        except Exception as e:
            print(f"Failed to create analysis session: {e}")
            return None
    
    def get_storage_stats(self) -> Dict:
        """Get sophisticated storage statistics"""
        return self.storage_stats.copy()

# ================================================================================
# REFACTORED MAIN PIPELINE WITH IN-MEMORY PROCESSING
# ================================================================================

# Initialize components
semantic_storage = SemanticRelationshipStorage(CONFIG["database"])
relationship_extractor = RelationshipExtractor()

def process_filing_with_pipeline(filing_data: Dict) -> Dict:
    """Process filing with in-memory entity and relationship extraction"""
    try:
        start_time = time.time()
        
        # Step 1: Extract sections (Cell 2 function)
        print(f"\n📄 Processing {filing_data['filing_type']} for {filing_data['company_domain']}")
        section_result = process_sec_filing_with_sections(filing_data)
        
        if section_result['processing_status'] != 'success':
            return {
                'success': False,
                'filing_id': filing_data.get('id'),
                'error': section_result.get('error', 'Section extraction failed'),
                'processing_time': time.time() - start_time
            }
        
        # Keep sections in memory for context retrieval
        sections_dict = section_result['sections']
        
        # Step 2: Extract entities (Cell 3 function) - keep in memory
        entities = entity_pipeline.process_sec_filing_sections(section_result)
        
        if not entities:
            return {
                'success': False,
                'filing_id': filing_data.get('id'),
                'error': 'No entities extracted',
                'processing_time': time.time() - start_time
            }
        
        print(f"   🔍 Extracted {len(entities)} entities")

        # Debug step removed
        # debug_entities call removed - function not defined

        # Step 3: Store entities IMMEDIATELY before Llama processing
        filing_ref = f"SEC_{filing_data.get('id')}"





        # Step 4: Extract relationships using in-memory entities and sections (LONG PROCESS)
        print(f"   🤖 Starting Llama 3.1 relationship extraction (this may take several minutes)...")
        
        # (relationships extraction continues below...)
        relationships = relationship_extractor.extract_company_relationships(
            entities, 
            sections_dict,
            filing_data['company_domain']
        )
        
        # Step 5: Store relationships using sophisticated schema
        filing_ref = f"SEC_{filing_data.get('id')}"
        if relationships:
            # Create analysis session for tracking
            session_id = semantic_storage.create_analysis_session(
                filing_data['company_domain'],
                [filing_ref]
            )
            # Store relationships with bucket aggregation
            relationship_storage_success = semantic_storage.store_relationships_with_buckets(
                relationships, filing_ref, session_id
            )
        else:
            relationship_storage_success = True
            print(f"   ℹ️ No relationships found to store")
        # Step 6: Verify storage

        
        processing_time = time.time() - start_time
        
        # Calculate overall storage success
        storage_success = True  # Storage removed - processing successful
        
        return {
            'success': storage_success,
            'filing_id': filing_data.get('id'),
            'company_domain': filing_data.get('company_domain'),
            'filing_type': filing_data.get('filing_type'),
            'sections_processed': len(sections_dict),
            'entities_extracted': len(entities),
            'relationships_found': len(relationships),
            'entities_stored': len(entities),
            'relationships_stored': len(relationships),
            'processing_time': round(processing_time, 2),
            'verification': {'entities': {'total': len(entities)}, 'relationships': {'total': len(relationships)}},
            'sample_entities': entities[:3],
            'sample_relationships': relationships[:3]
        }
        
    except Exception as e:
        return {
            'success': False,
            'filing_id': filing_data.get('id'),
            'error': str(e),
            'processing_time': time.time() - start_time
        }

def process_filings_batch(limit: int = 3) -> Dict:
    """Process multiple filings with complete in-memory pipeline"""
    print(f"\n🚀 Processing batch of {limit} SEC filings with in-memory pipeline...")
    
    batch_start = time.time()
    
    # Get unprocessed filings
    filings = get_unprocessed_filings(limit)
    
    if not filings:
        return {'success': False, 'message': 'No filings to process'}
    
    print(f"📊 Found {len(filings)} filings to process")
    
    # Process each filing
    results = []
    successful = 0
    total_entities = 0
    total_relationships = 0
    
    for i, filing in enumerate(filings, 1):
        print(f"\n[{i}/{len(filings)}] Processing {filing['filing_type']} for {filing['company_domain']}")
        
        result = process_filing_with_pipeline(filing)
        results.append(result)
        
        if result['success']:
            successful += 1
            total_entities += result.get('entities_extracted', 0)
            total_relationships += result.get('relationships_found', 0)
            
            print(f"   ✅ Success: {result['entities_extracted']} entities, {result['relationships_found']} relationships")
            print(f"   ⏱️ Processing time: {result['processing_time']}s")
            
            # Show sample relationships
            for rel in result.get('sample_relationships', [])[:2]:
                print(f"      • {rel['entity_text']} → {rel['relationship_type']} ({rel['business_impact']})")
        else:
            print(f"   ❌ Failed: {result.get('error', 'Unknown error')}")
        
        # Brief pause between filings
        if i < len(filings):
            time.sleep(1)
    
    batch_time = time.time() - batch_start
    
    # Update pipeline statistics
    entity_pipeline.pipeline_stats['documents_processed'] += successful
    entity_pipeline.pipeline_stats['total_entities_extracted'] += total_entities

    
    return {
        'success': successful > 0,
        'filings_processed': len(filings),
        'successful_filings': successful,
        'failed_filings': len(filings) - successful,
        'total_entities_extracted': total_entities,
        'total_relationships_found': total_relationships,
        'batch_processing_time': round(batch_time, 2),
        'avg_time_per_filing': round(batch_time / len(filings), 2) if filings else 0,
        'results': results
    }

# ================================================================================
# QUICK ACCESS FUNCTIONS
# ================================================================================

def test_pipeline(company_domain: str = None):
    """Test the pipeline with a single filing"""
    print("\n🧪 Testing in-memory pipeline...")
    
    # Get one filing
    if company_domain:
        # Modify get_unprocessed_filings to accept company filter
        # For now, just get any filing
        filings = get_unprocessed_filings(limit=1)
    else:
        filings = get_unprocessed_filings(limit=1)
    
    if not filings:
        print("❌ No test filings available")
        return None
    
    result = process_filing_with_pipeline(filings[0])
    
    if result['success']:
        print(f"\n✅ Pipeline test successful!")
        print(f"   📊 Sections: {result['sections_processed']}")
        print(f"   🔍 Entities: {result['entities_extracted']}")
        print(f"   🔗 Relationships: {result['relationships_found']}")
        print(f"   💾 Stored: {result['entities_stored']} entities, {result['relationships_stored']} relationships")
        print(f"   ⏱️ Time: {result['processing_time']}s")
    else:
        print(f"\n❌ Pipeline test failed: {result.get('error')}")
    
    return result

print("\n✅ In-Memory Pipeline Components Ready!")
print("   🎯 RelationshipExtractor with Llama 3.1")
print("   💾 Atomic storage for entities + relationships")
print("   🚀 In-memory processing (no DB round-trips)")
print("   📊 Usage: batch_results = process_filings_batch(limit=5)")
print("   🧪 Test: test_result = test_pipeline()")

def generate_pipeline_analytics_report() -> None:
    """Generate comprehensive analytics report for the pipeline"""
    print("\n" + "="*80)
    print("📊 ENTITYEXTRACTIONPIPELINE ANALYTICS DASHBOARD")
    print("="*80)
    
    # Database overview with enhanced metrics
    try:
        conn = psycopg2.connect(**NEON_CONFIG)
        cursor = conn.cursor()
        
        # Enhanced database statistics
        cursor.execute("""
            SELECT 
                COUNT(*) as total_entities,
                COUNT(DISTINCT company_domain) as companies,
                COUNT(DISTINCT sec_filing_ref) as filings,
                COUNT(DISTINCT entity_category) as entity_types,
                AVG(confidence_score) as avg_confidence,
                COUNT(*) FILTER (WHERE is_merged = true) as merged_entities,
                COUNT(DISTINCT primary_model) as active_models,
                COUNT(DISTINCT section_name) as sections_processed,
                COUNT(*) FILTER (WHERE section_name IS NOT NULL AND section_name != '') as entities_with_sections,
                MAX(extraction_timestamp) as last_extraction
            FROM system_uno.sec_entities_raw
            WHERE data_source = 'sec_filings'
        """)
        
        db_overview = cursor.fetchone()
        
        if db_overview and db_overview[0] > 0:
            total_entities = db_overview[0]
            entities_with_sections = db_overview[8]
            section_success_rate = (entities_with_sections / total_entities * 100) if total_entities > 0 else 0
            
            print(f"\n📈 DATABASE OVERVIEW:")
            print(f"   Total Entities Extracted: {db_overview[0]:,}")
            print(f"   Companies Processed: {db_overview[1]:,}")
            print(f"   SEC Filings Analyzed: {db_overview[2]:,}")
            print(f"   Entity Categories Found: {db_overview[3]:,}")
            print(f"   Average Confidence Score: {db_overview[4]:.3f}")
            print(f"   Multi-Model Entities: {db_overview[5]:,} ({db_overview[5]/db_overview[0]*100:.1f}%)")
            print(f"   Active Models: {db_overview[6]:,}")
            print(f"   Unique Sections Found: {db_overview[7]:,}")
            print(f"   🎯 SECTION SUCCESS RATE: {entities_with_sections:,}/{total_entities:,} ({section_success_rate:.1f}%)")
            print(f"   Last Extraction: {db_overview[9] or 'Never'}")
            
            # Alert if section success rate is low
            if section_success_rate < 90 and total_entities > 10:
                print(f"   🚨 WARNING: Section success rate is {section_success_rate:.1f}% - Pipeline routing issue!")
            elif section_success_rate >= 90:
                print(f"   ✅ EXCELLENT: Section success rate is {section_success_rate:.1f}% - Pipeline working correctly!")
        else:
            print(f"\n📈 DATABASE OVERVIEW: No entities found - database is clean for testing")
        
        cursor.close()
        conn.close()
                
    except Exception as e:
        print(f"   ❌ Could not retrieve analytics: {e}")
    
    # Pipeline statistics
    try:
        pipeline_stats = entity_pipeline.get_pipeline_statistics()
        
        print(f"\n🔧 PIPELINE STATISTICS:")
        print(f"   Documents Processed: {pipeline_stats['pipeline_stats']['documents_processed']:,}")
        print(f"   Total Entities Found: {pipeline_stats['pipeline_stats']['total_entities_extracted']:,}")
        print(f"   Processing Time: {pipeline_stats['pipeline_stats']['processing_time_total']:.2f}s")
        print(f"   Device: {pipeline_stats['device']}")
        print(f"   Loaded Models: {', '.join(pipeline_stats['loaded_models'])}")
        print(f"   Supported Sources: {', '.join(pipeline_stats['supported_data_sources'])}")
        
        # Individual model statistics
        print(f"\n📊 INDIVIDUAL MODEL PERFORMANCE:")
        for model_name, stats in pipeline_stats['model_stats'].items():
            if stats['texts_processed'] > 0:
                entities_per_text = stats['entities_found'] / stats['texts_processed']
                avg_time = stats['processing_time'] / stats['texts_processed']
                print(f"   {model_name:>12}: {stats['texts_processed']:>4} texts | {stats['entities_found']:>5} entities | {entities_per_text:>4.1f} avg/text | {avg_time:>4.2f}s avg")
        
        # Storage statistics


            print(f"\n💾 STORAGE STATISTICS:")
            print(f"   Entities Stored: {storage_stats['total_entities_stored']:,}")
            print(f"   Filings Processed: {storage_stats['filings_processed']:,}")
            print(f"   Merged Entities: {storage_stats['merged_entities']:,}")
            print(f"   Single-Model Entities: {storage_stats['single_model_entities']:,}")
            print(f"   Failed Inserts: {storage_stats['failed_inserts']:,}")
            
            merge_rate = (storage_stats['merged_entities'] / storage_stats['total_entities_stored'] * 100) if storage_stats['total_entities_stored'] > 0 else 0
            print(f"   Multi-Model Detection Rate: {merge_rate:.1f}%")
    
    except Exception as e:
        print(f"\n🔧 Pipeline statistics unavailable: {e}")
    
    print("\n" + "="*80)
    print("✅ EntityExtractionPipeline Analytics Complete!")
    print("="*80)


In [ ]:
# Cell 5: Main Processing Pipeline with Relationship Extraction

print("="*80)
print("🚀 STARTING SEC FILING PROCESSING PIPELINE")
print("="*80)

# Configure processing parameters
# CONFIG["processing"]["filing_batch_size"] = 3  # REMOVED: Use CONFIG["processing"]["filing_batch_size"] instead  # Number of filings to process in this run
# CONFIG["processing"]["enable_relationships"] = True  # REMOVED: Use CONFIG["processing"]["enable_relationships"] instead  # Set to False to skip relationship extraction

# Llama 3.1-8B is loaded locally, no API key needed
print("📝 Relationship extraction enabled with local Llama 3.1-8B")

# # Check if we have a Groq API key for relationship extraction
print("   ℹ️ Using local Llama 3.1-8B for relationship extraction")
#     print("   To enable relationships, add GROQ_API_KEY to Kaggle secrets")

# Check for available unprocessed filings
print("\n📊 Checking for unprocessed filings...")
available_filings = get_unprocessed_filings(limit=CONFIG["processing"]["filing_query_limit"])
print(f"   Found {len(available_filings)} unprocessed filings")

if available_filings:
    print("\n📋 Available filings to process:")
    for i, filing in enumerate(available_filings[:5], 1):
        print(f"   {i}. {filing['company_domain']} - {filing['filing_type']} ({filing['filing_date']})")
    
    # Process the batch
    print(f"\n🔄 Processing {min(CONFIG["processing"]["filing_batch_size"], len(available_filings))} filings...")
    print("-"*60)
    
    # Run the pipeline
    batch_results = process_filings_batch(limit=CONFIG["processing"]["filing_batch_size"])
    
    # Display results summary
    print("\n" + "="*80)
    print("📊 PROCESSING SUMMARY")
    print("="*80)
    
    if batch_results['success']:
        print(f"✅ Successfully processed {batch_results['successful_filings']}/{batch_results['filings_processed']} filings")
        print(f"   • Total entities extracted: {batch_results['total_entities_extracted']:,}")
        print(f"   • Total relationships found: {batch_results['total_relationships_found']:,}")
        print(f"   • Total processing time: {batch_results['batch_processing_time']:.1f}s")
        print(f"   • Average time per filing: {batch_results['avg_time_per_filing']:.1f}s")
        
        # Show detailed results for each filing
        print(f"\n📈 Detailed Results:")
        for i, result in enumerate(batch_results['results'], 1):
            if result['success']:
                print(f"\n   Filing {i}: {result['company_domain']} - {result['filing_type']}")
                print(f"      ✓ Sections: {result['sections_processed']}")
                print(f"      ✓ Entities: {result['entities_extracted']}")
                print(f"      ✓ Relationships: {result['relationships_found']}")
                print(f"      ✓ Time: {result['processing_time']:.1f}s")
            else:
                print(f"\n   Filing {i}: FAILED - {result.get('error', 'Unknown error')}")
        
        # Show pipeline statistics
        print(f"\n📊 Pipeline Statistics:")
        print(f"   • Documents processed (total): {entity_pipeline.pipeline_stats['documents_processed']}")
        print(f"   • Entities extracted (total): {entity_pipeline.pipeline_stats['total_entities_extracted']}")
        print(f"   • Storage transactions: {pipeline_storage.storage_stats['transactions_completed']} successful, {pipeline_storage.storage_stats['transactions_failed']} failed")
        print(f"   • Merged entities: {pipeline_storage.storage_stats['merged_entities']}")
        print(f"   • Single-model entities: {pipeline_storage.storage_stats['single_model_entities']}")
        
    else:
        print(f"❌ Processing failed: {batch_results.get('message', 'Unknown error')}")
    
    # Generate analytics report
    print("\n" + "="*80)
    generate_pipeline_analytics_report()
    
else:
    print("\n⚠️ No unprocessed filings found in raw_data.sec_filings")
    print("   All available filings have already been processed")
    print("\n💡 To add new filings:")
    print("   1. Insert new records into raw_data.sec_filings with accession_number")
    print("   2. Make sure the accession_number is valid (20 characters)")
    print("   3. Run this cell again to process them")

print("\n✅ Pipeline execution complete!")

In [ ]:
# Cell 6: Execute Pipeline

print(f"\n🎯 PRODUCTION COMMANDS:")
print(f"   • Process new filings: batch_results = process_filings_batch(limit=5)")
print(f"   • Check results:       generate_pipeline_analytics_report()")
print(f"   • View statistics:     context_retriever.get_retrieval_statistics()")

print(f"\n✅ EntityExtractionPipeline Production Interface Ready!")
print(f"🔧 SINGLE ENTRY POINT: process_filings_batch() - ensures all extractions use section-based pipeline")
print(f"📊 Database cleared - ready for fresh testing with guaranteed section extraction!")

In [ ]:
# Cell 6: TEST ENTITY STORAGE - Quick Verification Without Full Pipeline
# This cell tests ONLY the storage functionality with mock entities

print("="*80)
print("🧪 ENTITY STORAGE TEST - Verify Storage Works Without 90-Minute Run")
print("="*80)

# Create mock entities that match the expected structure
print("\n📝 Creating mock entities for storage test...")

# Create test entities with ALL required fields
test_entities = [
    {
        'extraction_id': str(uuid.uuid4()),
        'company_domain': 'test.com',
        'entity_text': 'Test Company Inc',
        'entity_type': 'ORGANIZATION',
        'entity_category': 'ORGANIZATION',  # Some code uses entity_category
        'confidence_score': 0.95,
        'char_start': 100,
        'char_end': 117,
        'character_start': 100,  # Some code uses character_start
        'character_end': 117,
        'surrounding_text': 'This is Test Company Inc in the context',
        'models_detected': ['bert_base', 'roberta'],
        'all_confidences': {'bert_base': 0.94, 'roberta': 0.96},
        'primary_model': 'roberta',
        'entity_variations': {'bert_base': 'Test Company Inc', 'roberta': 'Test Company Inc.'},
        'is_merged': True,
        'section_name': 'test_section',
        'data_source': 'sec_filings',
        'extraction_timestamp': datetime.now().isoformat(),
        'original_label': 'ORG',
        'model_source': 'roberta',  # Fallback field
        'quality_score': 0.92,
        'consensus_count': 2,
        'detecting_models': ['bert_base', 'roberta'],
        'consensus_score': 0.95,
        'filing_id': 999999,
        'sec_filing_ref': 'SEC_999999'
    },
    {
        'extraction_id': str(uuid.uuid4()),
        'company_domain': 'test.com',
        'entity_text': 'John Smith',
        'entity_type': 'PERSON',
        'entity_category': 'PERSON',
        'confidence_score': 0.88,
        'char_start': 200,
        'char_end': 210,
        'character_start': 200,
        'character_end': 210,
        'surrounding_text': 'CEO John Smith announced',
        'models_detected': ['bert_base'],
        'all_confidences': {'bert_base': 0.88},
        'primary_model': 'bert_base',
        'entity_variations': {'bert_base': 'John Smith'},
        'is_merged': False,
        'section_name': 'test_section',
        'data_source': 'sec_filings',
        'extraction_timestamp': datetime.now().isoformat(),
        'original_label': 'PER',
        'model_source': 'bert_base',
        'quality_score': 0.85,
        'consensus_count': 1,
        'detecting_models': ['bert_base'],
        'consensus_score': 0.88,
        'filing_id': 999999,
        'sec_filing_ref': 'SEC_999999'
    },
    {
        'extraction_id': str(uuid.uuid4()),
        'company_domain': 'test.com',
        'entity_text': 'FDA',
        'entity_type': 'ORGANIZATION',
        'entity_category': 'ORGANIZATION',
        'confidence_score': 0.99,
        'char_start': 300,
        'char_end': 303,
        'character_start': 300,
        'character_end': 303,
        'surrounding_text': 'approved by the FDA for clinical',
        'models_detected': ['bert_base', 'roberta', 'biobert'],
        'all_confidences': {'bert_base': 0.98, 'roberta': 0.99, 'biobert': 1.0},
        'primary_model': 'biobert',
        'entity_variations': {'bert_base': 'FDA', 'roberta': 'FDA', 'biobert': 'FDA'},
        'is_merged': True,
        'section_name': 'test_section',
        'data_source': 'sec_filings',
        'extraction_timestamp': datetime.now().isoformat(),
        'original_label': 'ORG',
        'model_source': 'biobert',
        'quality_score': 0.99,
        'consensus_count': 3,
        'detecting_models': ['bert_base', 'roberta', 'biobert'],
        'consensus_score': 0.99,
        'filing_id': 999999,
        'sec_filing_ref': 'SEC_999999'
    }
]

print(f"✅ Created {len(test_entities)} test entities")
print("\n📊 Test entity details:")
for i, entity in enumerate(test_entities, 1):
    print(f"   {i}. {entity['entity_text']} ({entity['entity_type']}) - confidence: {entity['confidence_score']:.2f}")

# Test storage
print("\n💾 Testing entity storage...")
print("   Using PipelineEntityStorage from Cell 4...")

try:
    # Initialize storage (assuming NEON_CONFIG is available from Cell 0)
    test_storage = PipelineEntityStorage(NEON_CONFIG)
    print("   ✅ Storage initialized successfully")
    
    # Attempt to store test entities
    filing_ref = "SEC_TEST_999999"
    print(f"\n   📤 Attempting to store {len(test_entities)} entities...")
    
    success = test_storage.store_entities(test_entities, filing_ref)
    
    if success:
        print("   ✅ STORAGE SUCCESSFUL! Entities stored to database")
        
        # Verify by querying the database
        print("\n   🔍 Verifying stored entities...")
        conn = psycopg2.connect(**NEON_CONFIG)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT COUNT(*), 
                   COUNT(DISTINCT entity_text),
                   AVG(confidence_score)::numeric(4,3),
                   AVG(quality_score)::numeric(4,3),
                   AVG(consensus_count)
            FROM system_uno.sec_entities_raw
            WHERE sec_filing_ref = %s
        """, (filing_ref,))
        
        result = cursor.fetchone()
        if result and result[0] > 0:
            print(f"   ✅ VERIFIED: {result[0]} entities found in database")
            print(f"      • Unique entities: {result[1]}")
            print(f"      • Avg confidence: {result[2]}")
            print(f"      • Avg quality: {result[3]}")
            print(f"      • Avg consensus: {result[4]}")
        else:
            print("   ❌ WARNING: Entities not found in database after storage")
        
        # Clean up test data
        print("\n   🧹 Cleaning up test data...")
        cursor.execute("DELETE FROM system_uno.sec_entities_raw WHERE sec_filing_ref = %s", (filing_ref,))
        conn.commit()
        deleted = cursor.rowcount
        print(f"   ✅ Cleaned up {deleted} test entities")
        
        cursor.close()
        conn.close()
        
    else:
        print("   ❌ STORAGE FAILED! Check error messages above")
        print("   ⚠️  The entity storage is NOT working correctly")
        
except Exception as e:
    print(f"   ❌ ERROR during storage test: {e}")
    print("   ⚠️  This error needs to be fixed before running the full pipeline")
    traceback.print_exc()

print("\n" + "="*80)
print("🏁 STORAGE TEST COMPLETE")
print("="*80)
print("\n📝 Summary:")
print("   • If you see '✅ STORAGE SUCCESSFUL', the storage is working")
print("   • If you see '❌ STORAGE FAILED', check the error and fix before running full pipeline")
print("   • This test takes <10 seconds vs 90 minutes for full pipeline")
